<a href="https://colab.research.google.com/github/GDevigili/TCC-DataWarehouse/blob/load-ibge-localidades/localidades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json
import pandas as pd

# Data Loading

In [2]:
url = "https://servicodados.ibge.gov.br/api/v1/localidades/distritos"
request = requests.get(url)
info = request.json()

Info é uma lista de dicionários que vem de `request.json()`

### sample data

In [3]:
info[:2]

[{'id': 520005005,
  'nome': 'Abadia de Goiás',
  'municipio': {'id': 5200050,
   'nome': 'Abadia de Goiás',
   'microrregiao': {'id': 52010,
    'nome': 'Goiânia',
    'mesorregiao': {'id': 5203,
     'nome': 'Centro Goiano',
     'UF': {'id': 52,
      'sigla': 'GO',
      'nome': 'Goiás',
      'regiao': {'id': 5, 'sigla': 'CO', 'nome': 'Centro-Oeste'}}}},
   'regiao-imediata': {'id': 520001,
    'nome': 'Goiânia',
    'regiao-intermediaria': {'id': 5201,
     'nome': 'Goiânia',
     'UF': {'id': 52,
      'sigla': 'GO',
      'nome': 'Goiás',
      'regiao': {'id': 5, 'sigla': 'CO', 'nome': 'Centro-Oeste'}}}}}},
 {'id': 310010405,
  'nome': 'Abadia dos Dourados',
  'municipio': {'id': 3100104,
   'nome': 'Abadia dos Dourados',
   'microrregiao': {'id': 31019,
    'nome': 'Patrocínio',
    'mesorregiao': {'id': 3105,
     'nome': 'Triângulo Mineiro/Alto Paranaíba',
     'UF': {'id': 31,
      'sigla': 'MG',
      'nome': 'Minas Gerais',
      'regiao': {'id': 3, 'sigla': 'SE', 'nome

In [4]:
info[0]

{'id': 520005005,
 'nome': 'Abadia de Goiás',
 'municipio': {'id': 5200050,
  'nome': 'Abadia de Goiás',
  'microrregiao': {'id': 52010,
   'nome': 'Goiânia',
   'mesorregiao': {'id': 5203,
    'nome': 'Centro Goiano',
    'UF': {'id': 52,
     'sigla': 'GO',
     'nome': 'Goiás',
     'regiao': {'id': 5, 'sigla': 'CO', 'nome': 'Centro-Oeste'}}}},
  'regiao-imediata': {'id': 520001,
   'nome': 'Goiânia',
   'regiao-intermediaria': {'id': 5201,
    'nome': 'Goiânia',
    'UF': {'id': 52,
     'sigla': 'GO',
     'nome': 'Goiás',
     'regiao': {'id': 5, 'sigla': 'CO', 'nome': 'Centro-Oeste'}}}}}}

Existem localidades repetidas, necessário limpeza disso

In [5]:
for i in range(5):
  print(info[i]['municipio'])

{'id': 5200050, 'nome': 'Abadia de Goiás', 'microrregiao': {'id': 52010, 'nome': 'Goiânia', 'mesorregiao': {'id': 5203, 'nome': 'Centro Goiano', 'UF': {'id': 52, 'sigla': 'GO', 'nome': 'Goiás', 'regiao': {'id': 5, 'sigla': 'CO', 'nome': 'Centro-Oeste'}}}}, 'regiao-imediata': {'id': 520001, 'nome': 'Goiânia', 'regiao-intermediaria': {'id': 5201, 'nome': 'Goiânia', 'UF': {'id': 52, 'sigla': 'GO', 'nome': 'Goiás', 'regiao': {'id': 5, 'sigla': 'CO', 'nome': 'Centro-Oeste'}}}}}
{'id': 3100104, 'nome': 'Abadia dos Dourados', 'microrregiao': {'id': 31019, 'nome': 'Patrocínio', 'mesorregiao': {'id': 3105, 'nome': 'Triângulo Mineiro/Alto Paranaíba', 'UF': {'id': 31, 'sigla': 'MG', 'nome': 'Minas Gerais', 'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}}, 'regiao-imediata': {'id': 310061, 'nome': 'Monte Carmelo', 'regiao-intermediaria': {'id': 3111, 'nome': 'Uberlândia', 'UF': {'id': 31, 'sigla': 'MG', 'nome': 'Minas Gerais', 'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}}}
{'id'

In [6]:
m_ = info[0]

# Data Unpacking

In [21]:
def unpack_json(json_dict: dict, row: dict = {}, parent_col = '') -> dict:
  """
    Make a documentation (how I wish colab had copilot :/ )
  """
  # Iterate over each key/value pair on the data
  for key in json_dict:
    # If the value is another dict, i.e. it has more data
    if type(json_dict[key]) == dict:
      # recursivelly call unpack_json() to extract the data
      # passing the a json_dict, the current row (pointer), and the name of the parent col
      unpack_json(json_dict[key], row = row, parent_col = key)
    else:
      # If it's a value (e.g. str, int, etc.), register it on the row
      # formatting the row's column name
      if parent_col == '':
        col_name = str(parent_col) + str(key)
      else:
        col_name = str(parent_col) + '_' + str(key)
      # register the value in the row
      row[col_name] = json_dict[key]
  return row.copy()

In [22]:
rows = []
for i in range(len(info)):
  data = unpack_json(info[i])
  rows.append(data)

In [23]:
df = pd.DataFrame(rows)

In [24]:
df

,id,nome,municipio_id,municipio_nome,microrregiao_id,microrregiao_nome,mesorregiao_id,mesorregiao_nome,UF_id,UF_sigla,UF_nome,regiao_id,regiao_sigla,regiao_nome,regiao-imediata_id,regiao-imediata_nome,regiao-intermediaria_id,regiao-intermediaria_nome
0,520005005,Abadia de Goiás,5200050,Abadia de Goiás,52010,Goiânia,5203,Centro Goiano,52,GO,Goiás,5,CO,Centro-Oeste,520001,Goiânia,5201,Goiânia
1,310010405,Abadia dos Dourados,3100104,Abadia dos Dourados,31019,Patrocínio,3105,Triângulo Mineiro/Alto Paranaíba,31,MG,Minas Gerais,3,SE,Sudeste,310061,Monte Carmelo,3111,Uberlândia
2,520010005,Abadiânia,5200100,Abadiânia,52012,Entorno de Brasília,5204,Leste Goiano,52,GO,Goiás,5,CO,Centro-Oeste,520002,Anápolis,5201,Goiânia
3,520010010,Posse d'Abadia,5200100,Abadiânia,52012,Entorno de Brasília,5204,Leste Goiano,52,GO,Goiás,5,CO,Centro-Oeste,520002,Anápolis,5201,Goiânia
4,310020305,Abaeté,3100203,Abaeté,31024,Três Marias,3106,Central Mineira,31,MG,Minas Gerais,3,SE,Sudeste,310070,Abaeté,3113,Divinópolis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10665,293360415,Iguira,2933604,Xique-Xique,29006,Barra,2902,Vale São-Franciscano da Bahia,29,BA,Bahia,2,NE,Nordeste,290021,Xique-Xique - Barra,2907,Irecê
10666,251740705,Zabelê,2517407,Zabelê,25010,Cariri Ocidental,2502,Borborema,25,PB,Paraíba,2,NE,Nordeste,250007,Monteiro,2502,Campina Grande
10667,355715405,Zacarias,3557154,Zacarias,35004,São José do Rio Preto,3501,São José do Rio Preto,35,SP,São Paulo,3,SE,Sudeste,350023,Birigui - Penápolis,3506,Araçatuba
10668,211400705,Zé Doca,2114007,Zé Doca,21008,Pindaré,2102,Oeste Maranhense,21,MA,Maranhão,2,NE,Nordeste,210009,Santa Inês,2102,Santa Inês - Bacabal
